Aqui temos a instalação das bibliotecas necessárias

In [1]:
!pip install crewai crewai_tools langchain_community
import warnings
warnings.filterwarnings('ignore')

Importando as classes Agent, Task e Crew, as quais são respectivamente responsáveis por nos permitir criar nossos agentes, criar as tarefas que nossos agentes vão desempenhar e por fim realizar a orquestração. LLM foi inserido pois não estou conseguindo usar meus créditos da OpenAI.

In [2]:
from crewai import Agent, Task, Crew, LLM

LLM que vai se conectar aos nosso agente e credenciais, pelo fato de não conseguir usar a da OpenAI vou usar minhas credenciais do gemini da google e devido a isso vou inserir, o parâmetro llm em todos os agentes para que eles usem o modelo gemini.

In [3]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.environ["GEMINI_API_KEY"]
            )

Abaixo temos nosso primeiro agente o planner, o qual conta com a classe Agent definida anteriormente, a qual conta com os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é ser preciso sobre determinado tema e uma história de fundo que gera contexto para o agente. Além disso temos os parâmetros allow_delegation e verbose, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

Abaixo temos nosso segundo agente o writter, o qual é bastante similar ao primeiro pois conta com a classe Agent definida, a qual possui os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é escrever sobre o assunto de maneira a gerar insights e ser bastante precisos, já o backstory se trata de uma história de fundo que gera contexto para o agente detalhando mais como ele deve agir e se portar. Além disso temos os parâmetros allow_delegation e verbose assim como no agente anterior, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

E por fim temos temos nosso terceiro agente o editor, o qual é bastante similar aos anteriores pois conta com a classe Agent definida, a qual possui os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é escrever sobre o assunto do blog de maneira organizada, já o backstory se trata de uma história de fundo que gera contexto para o agente detalhando mais como ele deve agir e se portar. Além disso temos os parâmetros allow_delegation e verbose assim como no agente anterior, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

Abaixo temos a a variável plan, a qual é uma das tasks que nossos agentes podem realizar, dessa forma temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual pode ser um arquivo .md, uma lista, uma tabela e etc., e o parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o planner criadfo anteriormente.

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

Abaixo temos a a variável write, a qual é uma das tasks que nossos agentes podem realizar e assim como a anterior temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual pode ser um arquivo .md, uma lista, uma tabela e etc., ou no caso dessa task um post em formato de markdow com 2 ou 3 paragráfos em cada seção, já parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o writer criado anteriormente.

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

E por fim abaixo temos a variável edite, a qual é uma das tasks que nossos agentes podem realizar e assim como a anterior temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual pode ser um arquivo .md, uma lista, uma tabela e etc., ou no caso dessa task um post em formato de markdow com 2 ou 3 paragráfos em cada seção, assim como a task anteior, já parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o editor criado anteriormente.

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Abaixo temos a variável crew, a qual recebe a classe Crew, onde temos os seguintes parâmetros agents, tasks e verbose, os quais respectivamente se tratam de uma lista de agentes os quais vão executar as tarefas, a lista de tarefas a serem executadas por nossos agentes e os pensamentos de nossos agentes ao executar as tarefas. Além vale destacar que nesse caso as tarefas são executadas sequencialmente, isto é plan -> write -> edit. Aqui diferente do tutorial a verbose teve que ser ajustado para true, devido as versões mais recentes da biblioteca.

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

Abaixo temos a variável results a qual recebe o metódo kickoff da variável crew, onde temos os inputs que se tratam de valores interpolados que podemos substituir nos agentes, ou seja, para todas as vezes que apraceu a palavra topic tanto na definição dos agentes ou das tasks podemos substituir por Artificial Intelligence e ao executar podemos observar a execução de cada tarefa uma por uma devido ao fato de verbose estar ativa, como declarado anteriormente.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6adb5c00-6433-45a0-a9b7-e8561fc6fd91                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 3a5fa1f1-2b6b-4fea-b960-041d9eb5e1fc
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 3a5fa1f1-2b6b-4fea-b960-041d9eb5e1fc
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: 3a5fa1f1-2b6b-4fea-b960-041d9eb5e1fc
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
# Decoding the AI Revolution: A Look into the Present and Future of Artificial Intelligence

**I. Introduction: A World Transformed by AI**

Artificial intelligence (AI) is rapidly transforming our world. From AI-generated artwork captivating the art world to AI-powered diagnostic tools revolutionizing healthcare, its impact is undeniable.  This article provides a clear overview of current AI trends, key players, and ethical considerations.  It aims to demystify AI for a broad audience, including tech enthusiasts, business professionals, and anyone curious about this transformative technology. We will explore AI's capabilities and limitations, addressing common concerns such as job displacement and algorithmic bias.

AI, broadly defined, encompasses various subfields including Machine Learning (algorithms enabling computers to learn from data), Deep Learning (a subset of machine learning using artificial neural networks), and Natural Language Process

🚀 Crew: crew
├── 📋 Task: 3a5fa1f1-2b6b-4fea-b960-041d9eb5e1fc
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: c5eb13df-a47f-4908-9b80-0026f292cc5e
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 0d5a254c-9045-4c54-885e-06471ae4582e
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 3a5fa1f1-2b6b-4fea-b960-041d9eb5e1fc
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: c5eb13df-a47f-4908-9b80-0026f292cc5e
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 0d5a254c-9045-4c54-885e-06471ae4582e
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0d5a254c-9045-4c54-885e-06471ae4582e                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6adb5c00-6433-45a0-a9b7-e8561fc6fd91                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

E por fim temos a exibição de tudo organizado

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

# Decoding the AI Revolution: A Look into the Present and Future of Artificial Intelligence

**I. Introduction: A World Transformed by AI**

Artificial intelligence (AI) is rapidly transforming our world. From AI-generated artwork captivating the art world to AI-powered diagnostic tools revolutionizing healthcare, its impact is undeniable.  This article provides a clear overview of current AI trends, key players, and ethical considerations.  It aims to demystify AI for a broad audience, including tech enthusiasts, business professionals, and anyone curious about this transformative technology. We will explore AI's capabilities and limitations, addressing common concerns such as job displacement and algorithmic bias.

AI, broadly defined, encompasses various subfields including Machine Learning (algorithms enabling computers to learn from data), Deep Learning (a subset of machine learning using artificial neural networks), and Natural Language Processing (allowing computers to understand and process human language). This exploration will navigate these complexities, offering a balanced perspective on AI's potential and challenges.


**II. Key AI Trends Shaping the Future: A Glimpse into Tomorrow**

Several key trends are shaping the future of AI. Generative AI, exemplified by tools like DALL-E 2, Midjourney, Stable Diffusion, and ChatGPT, is generating realistic images, art, and text.  These technologies hold immense potential across various sectors, but also raise ethical concerns regarding copyright and the potential for misuse (e.g., deepfakes).  The responsible development and deployment of these technologies are crucial.

AI's impact on healthcare is equally transformative. From accelerating drug discovery and improving diagnostic accuracy to enabling personalized medicine and assisting in robotic surgery, AI is revolutionizing medical practices.  While significant progress has been made, challenges remain regarding data privacy, algorithmic bias, and regulatory hurdles.

AI-driven automation is reshaping various industries, increasing efficiency in manufacturing, streamlining customer service, and enhancing fraud detection in finance.  However, the economic and social impact of this automation, particularly on employment, requires careful consideration and proactive strategies for workforce adaptation.

Edge AI, processing data closer to its source, is gaining traction due to its benefits in reducing latency, improving data privacy, and conserving bandwidth. This is driving innovation in autonomous vehicles and industrial IoT.


**III. Key Players and Their Impact: The Titans of AI**

Several key players are driving AI innovation. OpenAI, known for its large language models (GPT-3, GPT-4) and image generation model (DALL-E), is a significant force. Google DeepMind, with breakthroughs in reinforcement learning and AlphaFold (protein structure prediction), is another major contributor. Microsoft's investments in OpenAI and its Azure cloud platform significantly shape the AI landscape. Amazon Web Services (AWS) provides extensive AI-powered cloud services, while Meta focuses on computer vision and natural language processing. Nvidia's high-performance GPUs are crucial for many AI applications.  These major players, alongside numerous startups focusing on specialized applications, are shaping the industry.  [Links to relevant websites will be added in the final version.]


**IV. Ethical Considerations and Societal Impact: Navigating the Moral Maze**

The rapid advancement of AI necessitates careful consideration of its ethical implications. Algorithmic bias can lead to unfair or discriminatory outcomes. Automation's potential for job displacement requires proactive workforce adaptation strategies.  Concerns around data privacy are heightened by the vast amounts of data used to train AI models. Transparency, explainability, and accountability in AI systems are paramount to maintain public trust.

Regulations are emerging to promote responsible AI development, such as the EU AI Act.  These aim to mitigate risks and ensure AI benefits society. International collaboration is crucial in establishing global ethical guidelines.  The ongoing discussion around responsible AI governance highlights the complex societal challenges we face.


**V. Conclusion: The Future of AI - A Path Forward**

AI is rapidly evolving, presenting both opportunities and challenges. Generative AI, AI in healthcare, automation, and edge AI are transforming industries and impacting our lives.  Key players are constantly innovating, pushing technological boundaries.  However, ethical considerations surrounding bias, job displacement, and privacy must be addressed proactively. Responsible AI development, guided by ethical guidelines and regulations, is essential to harness its potential while mitigating risks, leading to a more equitable and beneficial society.


**VI. Call to Action: Join the Conversation!**

Share your thoughts on the future of AI in the comments section. What concerns you, and what excites you most?  Explore resources such as [insert links to relevant books, websites, and courses] to learn more. Subscribe to our blog for updates on AI developments.